In [1]:
#transfer learning model using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
#resource: https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub

from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image as Image
from tensorflow.keras import layers
import os
import zipfile
import cv2
from google.colab import drive
drive.mount('/content/drive')
print("success")
print(os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
success
/content


In [0]:
with zipfile.ZipFile("./drive/My Drive/train_and_test_images.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset/")

In [3]:
dim = (224, 224)
train_data_path = "./dataset/train_images"
engine_images = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
training_images = engine_images.flow_from_directory(str(train_data_path), target_size=dim)

test_data_path = "./dataset/test_images"
engine_images = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
testing_images = engine_images.flow_from_directory(str(test_data_path), target_size=dim)

Found 18635 images belonging to 120 classes.
Found 1945 images belonging to 120 classes.


In [5]:
#create neural network
#resource https://www.tensorflow.org/api_docs/python/tf/keras/Model

mobilenet = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                                                                                     input_shape=(224,224,3))
mobilenet.trainable = False #dont train the pretrained model

#create our complete NN with mobilenet connected new layer for our dog classes
model = tf.keras.Sequential([mobilenet, layers.Dense(training_images.num_classes, activation='softmax')])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_1 (Dense)              (None, 120)               153720    
Total params: 2,411,704
Trainable params: 153,720
Non-trainable params: 2,257,984
_________________________________________________________________


In [6]:
#training the new model
#resource: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
#resource: https://www.tensorflow.org/api_docs/python/tf/keras/losses

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

num_batches = np.ceil(training_images.samples/training_images.batch_size) #default batch size is 32 images

_ = model.fit(training_images, epochs=10, steps_per_epoch=num_batches, validation_data=testing_images)

Epoch 1/10
583/583 [==============================] - 103s 176ms/step - loss: 1.0859 - acc: 0.7178 - val_loss: 0.6419 - val_acc: 0.7871
Epoch 2/10
583/583 [==============================] - 92s 159ms/step - loss: 0.4245 - acc: 0.8696 - val_loss: 0.5868 - val_acc: 0.8129
Epoch 3/10
583/583 [==============================] - 93s 159ms/step - loss: 0.2831 - acc: 0.9163 - val_loss: 0.5782 - val_acc: 0.8154
Epoch 4/10
583/583 [==============================] - 92s 157ms/step - loss: 0.2020 - acc: 0.9454 - val_loss: 0.5788 - val_acc: 0.8134
Epoch 5/10
583/583 [==============================] - 92s 158ms/step - loss: 0.1489 - acc: 0.9643 - val_loss: 0.5843 - val_acc: 0.8201
Epoch 6/10
583/583 [==============================] - 92s 158ms/step - loss: 0.1110 - acc: 0.9771 - val_loss: 0.5813 - val_acc: 0.8231
Epoch 7/10
583/583 [==============================] - 91s 156ms/step - loss: 0.0855 - acc: 0.9840 - val_loss: 0.5849 - val_acc: 0.8231
Epoch 8/10
583/583 [==============================] - 

In [0]:
#The results show that we do not need so many epochs. After 3 epochs we are getting minimal improvement in accuracies and some decreases
#Since 3 epochs are sufficient, I am now going to retrain and save the model for only 3 epochs
#google colab is really cool. Lets you use gpu as well as increased ram

In [10]:
mobilenet = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                                                                                     input_shape=(224,224,3))
mobilenet.trainable = False
model = tf.keras.Sequential([mobilenet, layers.Dense(training_images.num_classes, activation='softmax')])
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
num_batches = np.ceil(training_images.samples/training_images.batch_size) #default batch size is 32 images
_ = model.fit(training_images, epochs=3, steps_per_epoch=num_batches, validation_data=testing_images)

Epoch 1/3
583/583 [==============================] - 97s 167ms/step - loss: 1.0906 - acc: 0.7159 - val_loss: 0.6449 - val_acc: 0.7974
Epoch 2/3
583/583 [==============================] - 94s 161ms/step - loss: 0.4241 - acc: 0.8698 - val_loss: 0.5904 - val_acc: 0.8077
Epoch 3/3
583/583 [==============================] - 93s 160ms/step - loss: 0.2863 - acc: 0.9160 - val_loss: 0.5812 - val_acc: 0.8139


In [16]:
print("I'm not sure why it is not saving. Should do it locally")
model.save("./drive/My Drive/dog_model_march_9", save_format='tf')

I'm not sure why it is not saving. Should do it locally


FailedPreconditionError: ignored